In [ ]:
import time
import argparse
import os
import pickle

import gym
from stable_baselines3 import PPO

import l5kit.environment
from l5kit.configs import load_config_data
from l5kit.data import LocalDataManager, ChunkedDataset, filter_agents_by_frames
from l5kit.dataset import EgoDataset
from l5kit.environment.envs.l5_env import SimulationConfigGym
from l5kit.environment.gym_metric_set import L2DisplacementYawMetricSet
from l5kit.environment.policy_utils import rollout_multiple_scenes
from l5kit.rasterization import build_rasterizer
from l5kit.visualization.visualizer.zarr_utils import episode_out_to_visualizer_scene_gym_cle
from l5kit.visualization.visualizer.visualizer import visualize
from l5kit.data import MapAPI

from bokeh.io import output_notebook, show
from prettytable import PrettyTable


In [ ]:
# set env variable for data
dm = LocalDataManager(None)
# get config
cfg = load_config_data("./gym_config.yaml")

mapAPI = MapAPI.from_cfg(dm, cfg)

env_config_path = './gym_config.yaml'

validation_sim_cfg = SimulationConfigGym()
validation_sim_cfg.num_simulation_steps = None
# validation_sim_cfg.distance_th_far = 500
# validation_sim_cfg.distance_th_close = 50
# validation_sim_cfg.disable_new_agents = True
env = gym.make("L5-CLE-v0", env_config_path=env_config_path, use_kinematic=True, return_info=True, train=True,
               sim_cfg=validation_sim_cfg)

output_notebook()

In [ ]:
env.sim_cfg


In [ ]:
def get_simulation_outputs(model_path, env, start=0, end=1):
    t_step = model_path.split('_')[-2]
    print("Time Step:", t_step)
    model = PPO.load(model_path, env)
    print("seed: ", model.seed)
    sim_outs = rollout_multiple_scenes(model, env, list(range(start, end)))
    return sim_outs

def visualize_outputs(sim_outs):
    for sim_out in sim_outs: # for each scene
        print("Scene ID: ", sim_out.scene_id)
        vis_in = episode_out_to_visualizer_scene_gym_cle(sim_out, mapAPI)
        show(visualize(sim_out.scene_id, vis_in))


def quantify_outputs(sim_outs, metric_set=None):
    metric_set = metric_set if metric_set is not None else L2DisplacementYawMetricSet()

    metric_set.evaluate(sim_outs)
    scene_results = metric_set.evaluator.scene_metric_results
    fields = ["scene_id", "FDE", "ADE"]
    table = PrettyTable(field_names=fields)
    tot_fde = 0.0
    tot_ade = 0.0
    for scene_id in scene_results:
        scene_metrics = scene_results[scene_id]
        ade_error = scene_metrics["displacement_error_l2"][1:].mean()
        fde_error = scene_metrics['displacement_error_l2'][-1]
        table.add_row([scene_id, round(fde_error.item(), 4), round(ade_error.item(), 4)])
        tot_fde += fde_error.item()
        tot_ade += ade_error.item()

    ave_fde = tot_fde / len(scene_results)
    ave_ade = tot_ade / len(scene_results)
    table.add_row(["Overall", round(ave_fde, 4), round(ave_ade, 4)])
    print(table)

from moviepy.editor import ImageSequenceClip
from IPython.display import display, Image

def gif_outputs(sim_outs, name='test'):
    sim_out = sim_outs[0] # for each scene
    print("Scene ID: ", sim_out.scene_id)
    vis_in = episode_out_to_visualizer_scene_gym_cle(sim_out, mapAPI)
    gif_frames = visualize_gif(sim_out.scene_id, vis_in)

    # save it as a gif
    clip = ImageSequenceClip(list(gif_frames), fps=20)
    clip.write_gif(name + '.gif', fps=20)

In [ ]:
model_file = './logs/verify_allscenes_20000000_steps'
model = PPO.load(model_file, env)

start = 0
end = 1
name = 'validate_0'
sim_outs = get_simulation_outputs(model_file, env, start=start, end=end)
quantify_outputs(sim_outs)
#     sim_outs = sim_outs[0:10]
#     visualize_outputs(sim_outs)
gif_outputs(sim_outs, name)


In [ ]:
model_file = './logs/verify_100scenes_4000000_steps'
model = PPO.load(model_file, env)

start = 0
end = 10
sim_outs = get_simulation_outputs(model_file, env, start=start, end=end)
quantify_outputs(sim_outs)
visualize_outputs(sim_outs)


In [ ]:
model_file = './logs/verify_16_1000000_steps'
model = PPO.load(model_file, env)

start = 0
end = 1
sim_outs = get_simulation_outputs(model_file, env, start=start, end=end)
quantify_outputs(sim_outs)
# visualize_outputs(sim_outs)

In [ ]:
model_file = './logs/verify_s10_8_deepcnn_800000_steps'
model = PPO.load(model_file, env)

start = 0
end = 10
sim_outs = get_simulation_outputs(model_file, env, start=start, end=end)
quantify_outputs(sim_outs)
visualize_outputs(sim_outs)

In [ ]:
model_file = './logs/verify_yaw45_s100_g080_yaw1_32_10000000_steps'
model = PPO.load(model_file, env)

start = 0
end = 10
sim_outs = get_simulation_outputs(model_file, env, start=start, end=end)
quantify_outputs(sim_outs)
visualize_outputs(sim_outs)

In [ ]:
model_file = './logs/verify_yaw45_s100_g080_yaw1_32_10000000_steps'
model = PPO.load(model_file, env)

start = 6
end = 10
sim_outs = get_simulation_outputs(model_file, env, start=start, end=end)
quantify_outputs(sim_outs)
visualize_outputs(sim_outs)

In [ ]:
model_file = './logs/verify_yaw45_s100_g080_yaw1_32_10000000_steps'
model = PPO.load(model_file, env)

start = 15
end = 20
sim_outs = get_simulation_outputs(model_file, env, start=start, end=end)
quantify_outputs(sim_outs)
visualize_outputs(sim_outs)

In [ ]:
model_file = './logs/verify_yaw60_s10_g080_yaw1_32_6000000_steps'
model = PPO.load(model_file, env)

start = 0
end = 2
sim_outs = get_simulation_outputs(model_file, env, start=start, end=end)
quantify_outputs(sim_outs)
visualize_outputs(sim_outs)

In [ ]:
validation_sim_cfg = SimulationConfigGym()
validation_sim_cfg.num_simulation_steps = None
validation_sim_cfg.use_agents_gt = True
env = gym.make("L5-CLE-v0", env_config_path=env_config_path, use_kinematic=True, return_info=True, train=False,
               sim_cfg=validation_sim_cfg, simnet_model_path="/code/parth/dataset/dn_h3_p05_1999999_steps.pt")
model_file = './logs/verify_yaw60_s10_g080_yaw1_32_5200000_steps'
model = PPO.load(model_file, env)

start = 2
end = 10
sim_outs = get_simulation_outputs(model_file, env, start=start, end=end)
quantify_outputs(sim_outs)
visualize_outputs(sim_outs)

In [ ]:
validation_sim_cfg = SimulationConfigGym()
validation_sim_cfg.num_simulation_steps = None
validation_sim_cfg.use_agents_gt = False
env = gym.make("L5-CLE-v0", env_config_path=env_config_path, use_kinematic=True, return_info=True, train=False,
               sim_cfg=validation_sim_cfg, simnet_model_path="/home/ubuntu/src/l5kit/examples/dro/checkpoints/drivenet_h0_p05_onecycle_schedule_step5_jit_r18_378720_steps.pt")
model_file = './logs/verify_yaw60_s10_g080_yaw1_32_5200000_steps'
model = PPO.load(model_file, env)

start = 0
end = 5

sim_outs = get_simulation_outputs(model_file, env, start=start, end=end)
quantify_outputs(sim_outs)
visualize_outputs(sim_outs)

In [ ]:
validation_sim_cfg = SimulationConfigGym()
validation_sim_cfg.num_simulation_steps = None
validation_sim_cfg.use_agents_gt = True
env = gym.make("L5-CLE-v0", env_config_path=env_config_path, use_kinematic=True, return_info=True, train=False,
               sim_cfg=validation_sim_cfg, simnet_model_path="/home/ubuntu/src/l5kit/examples/dro/checkpoints/drivenet_h0_p05_onecycle_schedule_step5_jit_r18_378720_steps.pt")
model_file = './logs/verify_yaw60_s10_g080_yaw1_32_5200000_steps'
model = PPO.load(model_file, env)

start = 0
end = 5

sim_outs = get_simulation_outputs(model_file, env, start=start, end=end)
quantify_outputs(sim_outs)
# visualize_outputs(sim_outs)

In [ ]:
model_file = './logs/verify_yaw60_s10_g080_yaw1_32_5200000_steps'
model = PPO.load(model_file, env)

start = 0
end = 1
sim_outs = get_simulation_outputs(model_file, env, start=start, end=end)
quantify_outputs(sim_outs)
visualize_outputs(sim_outs)

In [ ]:
model_file = './logs/verify_yaw60_s100_g080_yaw1_32_5000000_steps'
model = PPO.load(model_file, env)

start = 0
end = 5
sim_outs = get_simulation_outputs(model_file, env, start=start, end=end)
quantify_outputs(sim_outs)
visualize_outputs(sim_outs)

In [ ]:
model_file = './logs/verify_yaw45_s100_g080_yaw1_32_run3_75M_stepLR_3000000_steps'
model = PPO.load(model_file, env)

start = 0
end = 5
sim_outs = get_simulation_outputs(model_file, env, start=start, end=end)
quantify_outputs(sim_outs)
visualize_outputs(sim_outs)

In [ ]:
model_file = './logs/verify_yaw45_s100_g080_yaw1_32_sample_6500000_steps'
model = PPO.load(model_file, env)

start = 7
end = 9
sim_outs = get_simulation_outputs(model_file, env, start=start, end=end)
quantify_outputs(sim_outs)
visualize_outputs(sim_outs)

In [ ]:
from IPython.display import display, Image
Image(name + '.gif')